In [6]:
# !pip install konlpy
# !pip install sentence_transformers

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer # 단어 토큰화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도 계산
from sentence_transformers import SentenceTransformer   # 문장 임베딩을 위한 모델

c:\Users\user\.virtualenvs\project-D53q-EhT\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### Dataset load

In [5]:
df = pd.read_feather("../data/processing/stopWordsRemoved_total.feather", use_threads=True)
df

,Query,News,Title,Text
0,더불어민주당,asiae,"[막대기, 당선, 판진, 중권, 네거티브, 민주당, 비판]","[진중권, 전동, 양대, 교수, 이미지, 출처, 연합뉴스, 원본, 아이콘, 아시아,..."
1,더불어민주당,asiae,"[공수, 민주당, 사죄, 국민의힘, 여론, 조사, 민심]","[이낙연, 분노, 실망, 아프, 반성, 혁신, 주호영, 여론, 조사, 차이, 민심,..."
2,더불어민주당,asiae,"[종합, 민주당, 오세훈, 내곡동, 특혜, 의혹, 거짓말, 거짓말, 맹공]","[박영선, 토론, 오세훈, 내곡동, 특혜, 의혹, 공세, 서울, 시장, 보궐, 선거..."
3,더불어민주당,asiae,"[박형준, 안민석, 더불어민주당, 원진, 보유, 튜버, 부산, 지검, 고발]","[국민의힘, 캠프, 부동산, 투기, 전혀, 의원, 후보, 부인, 부동산, 복부인, ..."
4,더불어민주당,asiae,"[포토, 더불어민주당, 원내, 대책, 회의]","[가장, 많이, 뉴스, 제공, 집계, 기준, 최대, 전기사, 제공]"
...,...,...,...,...
181819,더불어민주당,YTN,"[여론, 톡톡, 김기현, 책임, 확산, 이낙연, 이준석, 손잡]","[진행, 김영수, 앵커, 출연, 김형준, 배재, 석좌, 교수, 배종찬, 인사이트, ..."
181820,더불어민주당,YTN,"[홍익표, 특검, 본회, 국정, 조사, 연내, 처리]","[더불어민주당, 홍익표, 원내대표, 특검, 법안, 예산안, 본회의, 올리, 국정, ..."
181821,더불어민주당,YTN,"[손학규, 병립형, 회귀, 후퇴, 이재명, 약속]","[손학규, 미래, 대표, 선거제, 개편, 논의, 관련, 연동, 비례제, 더불어민주당..."
181822,더불어민주당,YTN,"[민주, 송영길, 구속영장, 청구, 검찰, 독재, 정권, 총선, 전략]","[더불어민주당, 검찰, 이전, 대회, 봉투, 의혹, 연루, 송영길, 대표, 구속, ..."


In [119]:
doc = ' '.join(word for word in df['Text'][7])  
print(f"doc: {doc}")

doc: 국민의힘 캠프 부동산 투기 전혀 의원 후보 부인 부동산 복부인 발언 반박 부산 시장 보선 국민의힘 박형준 후보 오후 부산 시민 공원 민주당 부산 비하 발언 규탄 대회 시민 지지 호소 이미지 출처 연합뉴스 원본 아이콘 아시아 경제 영남 취재 본부 김용우 기자 박형준 국민의힘 부산 시장 후보 선대위 더불어민주당 안민석 의원 튜버 진보 성향 시민 단체 관계자 부산 지검 고발 후보 캠프 의원 유튜브 채널 서울 소리 운영 자백 종씨 대구 시민 운영 박대희 신원 미상 공직 선거법 허위 사실 공표 부산 지검 수사 의뢰 캠프 의원 제외 폭력 행위 처벌 법공 동주 거침 위반 혐의 추가 고발 안민석 의원 부산 북구 구포 시장 더불어민주당 합동 유세 부동산 복부인 부동산 투기꾼 시장 사모 부동산 투기 연설 캠프 유튜브 후보 건물 단층 건물 후보 건물 허위 사실 방송 허위 사실 공표 명예 훼손 주장 선대위 법률 원단 부산 시장 보선 앞두 민주당 소속 국회의원 일부 튜버 후보 당선 목적 허위 사실 공표 후보 배우자 비방 유권자 선택 오도 범죄 행위 주장 후보 캠프 후보 배우자 조모 부동산 투기 사실 전혀 의원 발언 공직 선거법 허위 사실 공표 해당 시민 많이 공공연히 허위 사실 연설 명예 훼손 반박 다영 남취 본부 김용우 기자 투자가 경제 콘텐츠 플랫 아시아 경제 단전 배포 금지


- Sckit-learn의 CountVectorizer를 사용해 단어 추출
- n_gram_range의 인자를 사용하면, 쉽게 n-gram을 추출할 수 있음
- 예를 들어 (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출

In [120]:
n_gram_range = (2, 3)   # n-gram 범위

count = CountVectorizer(ngram_range=n_gram_range)   # n-gram 벡터화
count.fit([doc])    # n-gram 벡터화를 위한 단어 사전 구축

CountVectorizer(ngram_range=(2, 3))

In [121]:
candidates = count.get_feature_names_out()  # n-gram 후보 추출
print(f"trigram 개수: {len(candidates)}")
print(f"trigram 5개만 출력: {candidates[:5]}")

trigram 개수: 365
trigram 5개만 출력: ['거침 위반' '거침 위반 혐의' '건물 단층' '건물 단층 건물' '건물 허위']


#### 문서와 문서로부터 추출한 키워들을 수치화
- 한국어를 포함하고 있는 다국어 SBERT 모델 사용

In [122]:
model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")  # 문장 임베딩 모델

c:\Users\user\.virtualenvs\project-D53q-EhT\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [123]:
doc_embedding = model.encode([doc]) # 문서 임베딩
candidate_embeddings = model.encode(candidates) # 후보군 임베딩

#### 문서와 가장 유사한 키워드 추출 (문서를 대표하기 좋은 키워드)
- top_n을 통해 상위 5개의 키워드를 출력

In [124]:
top_n = 10   # 상위 n개 후보군 선택
distances = cosine_similarity(doc_embedding, candidate_embeddings) # 코사인 유사도 계산
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]] # 상위 n개 후보군 선택

In [125]:
print(f"keywords: {keywords}")

keywords: ['반박 부산 시장', '민주당 부산 비하', '부산 북구 구포', '부산 지검 고발', '부산 지검 수사', '법률 원단 부산', '공표 부산 지검', '박형준 국민의힘 부산', '원단 부산 시장', '공원 민주당 부산']


In [141]:
# 정당: political_party
def extract_politicalParty(keywords):

    count_dict = {"더불어민주당": 0, "국민의힘": 0} # 정당별 카운트 딕셔너리
    for keyword in keywords:    # 키워드 리스트에서
        for word in keyword.split():    # 키워드를 공백 기준으로 분리하여
            if "국민의힘" in word:  # "국민의힘"이 포함되어 있으면
                count_dict["국민의힘"] += 1  # "국민의힘" 카운트 증가
                
            if "민주당" in word:  # "더불어민주당"이 포함되어 있으면
                count_dict["더불어민주당"] += 1 # "더불어민주당" 카운트 증가

    # print(f"count_dict: {count_dict}")
    if count_dict["더불어민주당"] == count_dict["국민의힘"]:
        return np.nan   # 두 정당이 동일하게 언급되었을 경우 np.nan 반환
    else:    # 더불어민주당과 국민의힘 중 더 많이 언급된 정당을 반환
        return max(count_dict, key=count_dict.get)  
    
print(f"정당: {extract_politicalParty(keywords)}")

정당: 국민의힘


In [127]:
doc

'국민의힘 캠프 부동산 투기 전혀 의원 후보 부인 부동산 복부인 발언 반박 부산 시장 보선 국민의힘 박형준 후보 오후 부산 시민 공원 민주당 부산 비하 발언 규탄 대회 시민 지지 호소 이미지 출처 연합뉴스 원본 아이콘 아시아 경제 영남 취재 본부 김용우 기자 박형준 국민의힘 부산 시장 후보 선대위 더불어민주당 안민석 의원 튜버 진보 성향 시민 단체 관계자 부산 지검 고발 후보 캠프 의원 유튜브 채널 서울 소리 운영 자백 종씨 대구 시민 운영 박대희 신원 미상 공직 선거법 허위 사실 공표 부산 지검 수사 의뢰 캠프 의원 제외 폭력 행위 처벌 법공 동주 거침 위반 혐의 추가 고발 안민석 의원 부산 북구 구포 시장 더불어민주당 합동 유세 부동산 복부인 부동산 투기꾼 시장 사모 부동산 투기 연설 캠프 유튜브 후보 건물 단층 건물 후보 건물 허위 사실 방송 허위 사실 공표 명예 훼손 주장 선대위 법률 원단 부산 시장 보선 앞두 민주당 소속 국회의원 일부 튜버 후보 당선 목적 허위 사실 공표 후보 배우자 비방 유권자 선택 오도 범죄 행위 주장 후보 캠프 후보 배우자 조모 부동산 투기 사실 전혀 의원 발언 공직 선거법 허위 사실 공표 해당 시민 많이 공공연히 허위 사실 연설 명예 훼손 반박 다영 남취 본부 김용우 기자 투자가 경제 콘텐츠 플랫 아시아 경제 단전 배포 금지'

---

In [128]:
sample = pd.read_pickle("../data/processing/deduplicated_total.pkl")
sample

,Query,Title,Text,News
0,더불어민주당,막대기도 당선될 판 진중권 네거티브 민주당 비판,진중권 전 동양대 교수 이미지출처연합뉴스 AD 원본보기 아이콘아시아경제 황수미 기자...,asiae
1,더불어민주당,1년만에 뒤바뀐 공수민주당 사죄 국민의힘 여론조사가 민심,이낙연 분노와 실망 아프도록 잘 안다 반성하고 혁신주호영 여론조사 2030 차이 민...,asiae
2,더불어민주당,종합민주당 오세훈 내곡동 특혜 의혹에 거짓말이 거짓말을 낳아 맹공,박영선 TV 토론서 오세훈 내곡동 특혜 의혹 두고 공세47 서울시장 보궐선거 더불어...,asiae
3,더불어민주당,박형준 안민석 더불어민주당 의원진보 유튜버 등 4명 부산지검 고발,국민의힘 캠프 부동산 투기 전혀 없다 안 의원 후보부인 부동산 복부인 발언에 반박...,asiae
4,더불어민주당,포토 더불어민주당 원내대책회의,가장 많이 읽힌 뉴스를 제공합니다 집계 기준에 따라 최대 3일 전 기사까지 제공될 ...,asiae
...,...,...,...,...
181819,더불어민주당,여론톡톡 김기현 책임론 확산이낙연이준석 손잡나,AD 진행 김영수 앵커 출연 김형준 배재대 석좌교수 배종찬 인사이트 K 연구소장...,YTN
181820,더불어민주당,홍익표 쌍특검 8일 본회의국정조사도 연내 처리,AD더불어민주당 홍익표 원내대표는 쌍특검 법안과 예산안은 오는 8일 본회의에 올리고...,YTN
181821,더불어민주당,손학규 병립형 회귀 큰 후퇴이재명 약속 지켜야,AD손학규 전 바른미래당 대표는 선거제 개편 논의와 관련해 연동형 비례제는 더불어민...,YTN
181822,더불어민주당,민주 송영길 구속영장 청구 검찰 독재정권의 총선 전략,AD더불어민주당은 검찰이 전당대회 돈봉투 의혹에 연루된 송영길 전 대표에게 구속영장...,YTN


In [156]:
for i in range(20):
    print(f"{i}: {sample.loc[i].Title}")

0: 막대기도 당선될 판 진중권 네거티브 민주당 비판
1: 1년만에 뒤바뀐 공수민주당 사죄 국민의힘 여론조사가 민심
2: 종합민주당 오세훈 내곡동 특혜 의혹에 거짓말이 거짓말을 낳아 맹공
3: 박형준 안민석 더불어민주당 의원진보 유튜버 등 4명 부산지검 고발
4: 포토 더불어민주당 원내대책회의
5: 선거 D9 4차 지원금 시작민주당 반전 계기 될까
6: 포토 사전투표 독려하는 민주당 지도부
7: 단독민주당 신복지 보고서 소득 없는 중장년에 1년간 월 100만원
8: 종합 박형준 딸 입시비리 논란하태경 아무말 대잔치 민주당 교육 적폐 밝혀야
9: 민주당 오세훈안철수 단일화 야합스스로를 높이려는 욕망으로 점철
10: 이준석 지역 다니면 안다민주당 조직력 소가 웃을 이야기
11: 민주당 보병전 조직의 힘으로 반전 노려
12: 진중권 이해찬은 민주당의 원흉현실 판단 안 되는 듯
13: 민주당 박형준 옹호한 하태경에 성 다르면 아들 아닌가 한심
14: 허은아 고민정 정신 못차려민주당 블루 때문에 국민은 피멍
15: 국회 패스트트랙 충돌 민주당 공판 또 연기5월로 늦춰져
16: 민주당 오세훈 내곡동 땅 분명히 인식 추가 증거 제출
17: 민주당 박형준 차도남 아닌 까도남김종인 결단해야
18: 박근철 경기의회 민주당 대표 기본주택은 투기 잠재울 수 있는 대안 
19:  투기의혹 전수조사 전원 동의민주당도 검증대 나와라


In [ ]:
# top_n = 5, 정확도: 7/17
- 0: 더불어민주당 - 국민의힘
- 1: ? - 국민의힘
- 2: 국민의힘 - 국민의힘
- 3: 더불어민주당 - nan
- 7: 더불어민주당 - nan
- 8: 더불어민주당 - 국민의힘
- 9: 국민의힘 - 국민의힘
- 10: 더불어민주당 - nan
- 11: 더불어민주당 - 더불어민주당
- 12: 더불어민주당 - 국민의힘
- 13: 더불어민주당 - 국민의힘
- 14: 더불어민주당 - 더불어민주당
- 15: 더불어민주당 - nan
- 16: 더불어민주당 - 더불어민주당
- 17: 국민의힘  - 국민의힘
- 18: 더불어민주당 - nan
- 19: 더불어민주당 - 더불어민주당

In [ ]:
0, 더불어민주당 - 국민의힘
1, ? -  국민의힘
2, 국민의힘: 국민의힘
3, 더불어민주당: nan
7, 더불어민주당: nan
8, 더불어민주당: 국민의힘
9, 국민의힘: 국민의힘
10, 더불어민주당: nan
11, 더불어민주당: 더불어민주당
12, 더불어민주당: 국민의힘
13, 더불어민주당: 국민의힘
14, 더불어민주당: 더불어민주당
15, 더불어민주당: nan
16, 더불어민주당: 더불어민주당
17, 국민의힘: 국민의힘
18, 더불어민주당: nan
19, 더불어민주당: 더불어민주당

In [ ]:
# top_n = 10, 정확도: 8/17
- 0: 더불어민주당 - 국민의힘
- 1: ? - 국민의힘
- 2: 국민의힘 - 국민의힘
- 3: 더불어민주당 - 더불어민주당
- 7: 더불어민주당 - nan
- 8: 더불어민주당 - 국민의힘
- 9: 국민의힘 - 국민의힘
- 10: 더불어민주당 - 더불어민주당
- 11: 더불어민주당 - 더불어민주당
- 12: 더불어민주당 - 국민의힘
- 13: 더불어민주당 - 국민의힘
- 14: 더불어민주당 - 더불어민주당
- 15: 더불어민주당 - 더불어민주당
- 16: 더불어민주당 - nan
- 17: 국민의힘 - 국민의힘
- 18: 더불어민주당 - nan
- 19: 더불어민주당 - nan

In [ ]:
# top_n = 15, 정확도: 6/17
- 0: 더불어민주당 - 국민의힘
- 1: ? - 국민의힘
- 2: 국민의힘 - 국민의힘
- 3: 더불어민주당 - 더불어민주당
- 7: 더불어민주당 - nan
- 8: 더불어민주당 - 국민의힘
- 9: 국민의힘 - nan
- 10: 더불어민주당 - 더불어민주당
- 11: 더불어민주당 - 더불어민주당
- 12: 더불어민주당 - 국민의힘
- 13: 더불어민주당 - 국민의힘
- 14: 더불어민주당 - 더불어민주당
- 15: 더불어민주당 - 더불어민주당
- 16: 더불어민주당 - 국민의힘
- 17: 국민의힘 - nan
- 18: 더불어민주당 - nan
- 19: 더불어민주당 - 국민의힘

# top_n = 20, 정확도: 5/10
- 0: 더불어민주당 - 국민의힘
- 1: ? - 국민의힘
- 2: 국민의힘 - 국민의힘
- 3: 더불어민주당 - 더불어민주당
- 7: 더불어민주당 - nan
- 8: 더불어민주당 - 국민의힘
- 9: 국민의힘 - nan
- 10: 더불어민주당 - 더불어민주당
- 11: 더불어민주당 - 더불어민주당
- 12: 더불어민주당 - 국민의힘
- 13: 더불어민주당 - 국민의힘
- 14: 더불어민주당 - 더불어민주당
- 15: 더불어민주당 - nan
- 16: 더불어민주당 - 국민의힘
- 17: 국민의힘 - nan
- 18: 더불어민주당 - nan
- 19: 더불어민주당 - 국민의힘

# top_n = 30 정확도: 7/17
- 0: 더불어민주당 - 국민의힘
- 1: ? - 국민의힘
- 2: 국민의힘 - 국민의힘
- 3: 더불어민주당 - nan
- 7: 더불어민주당 - nan
- 8: 더불어민주당 - 국민의힘
- 9: 국민의힘 - 국민의힘
- 10: 더불어민주당 - 더불어민주당
- 11: 더불어민주당 - 더불어민주당
- 12: 더불어민주당 - 국민의힘
- 13: 더불어민주당 - 국민의힘
- 14: 더불어민주당 - 더불어민주당
- 15: 더불어민주당 - nan
- 16: 더불어민주당 - 더불어민주당
- 17: 국민의힘 - nan
- 18: 더불어민주당 - 더불어민주당
- 19: 더불어민주당 - 국민의힘

In [180]:
for i in range(20):
    if i in [4, 5, 6]:
        continue
    doc = ' '.join(word for word in (df['Title'][i].tolist() + df["Text"][i].tolist()))  
    # print(f"doc: {doc}")

    n_gram_range = (2, 3)   # n-gram 범위

    count = CountVectorizer(ngram_range=n_gram_range)   # n-gram 벡터화
    count.fit([doc])    # n-gram 벡터화를 위한 단어 사전 구축

    candidates = count.get_feature_names_out()  # n-gram 후보 추출
    # print(f"trigram 개수: {len(candidates)}")
    # print(f"trigram 5개만 출력: {candidates[:5]}")

    # model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")  # 문장 임베딩 모델

    doc_embedding = model.encode([doc]) # 문서 임베딩
    candidate_embeddings = model.encode(candidates) # 후보군 임베딩

    top_n = 7   # 상위 n개 후보군 선택
    distances = cosine_similarity(doc_embedding, candidate_embeddings) # 코사인 유사도 계산
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]] # 상위 n개 후보군 선택

    # print()
    print(f"{i}, 정당: {extract_politicalParty(keywords)}")

0, 정당: 국민의힘
1, 정당: 국민의힘
2, 정당: 국민의힘
3, 정당: nan
7, 정당: nan
8, 정당: 국민의힘
9, 정당: 국민의힘
10, 정당: nan
11, 정당: 더불어민주당
12, 정당: 국민의힘
13, 정당: 국민의힘
14, 정당: 더불어민주당
15, 정당: nan
16, 정당: 더불어민주당
17, 정당: 국민의힘
18, 정당: nan
19, 정당: 더불어민주당
